In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import glob
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models
import urllib.request
import scipy.io as io
import h5py
import progressbar
from __future__ import print_function
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB5, Xception,ResNet50V2


In [ ]:
# Parameters
TIME_STEP = 256
VAL_SIZE = 0.22
NUM_FEATURES = 10
BATCH_SIZE = 64
EPOCHS = 64

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(data, target):
  feature = {
      'data': _float_feature(data),
      'target': _float_feature([target])
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()


In [ ]:
path_to_csv = '/content/drive/MyDrive/5min/BTCUSDT.csv'
data = pd.read_csv(path_to_csv)
data = data.drop(columns=['Close-time', 'Open-time'])
data.head()

,Open,High,Low,Close,Volume,Quote-asset-volume,Number-of-trades,Taker-buy-base-asset-volume,Taker-buy-quote-asset-volume,Can-be-ignored
0,4261.48,4280.56,4261.48,4261.48,2.189061,9333.620962,9,0.489061,2089.104962,7960.375295
1,4261.48,4261.48,4261.48,4261.48,0.000000,0.000000,0,0.000000,0.000000,7959.626292
2,4261.48,4261.48,4261.48,4261.48,0.000000,0.000000,0,0.000000,0.000000,7958.417415
3,4261.48,4264.88,4261.48,4261.48,0.484666,2065.447829,9,0.085455,364.218136,7958.348965
4,4264.88,4266.29,4264.88,4266.29,2.328570,9931.161124,11,1.546491,6595.688039,7953.969349


In [ ]:
data = np.array(data)
data.shape

(453594, 10)

In [ ]:
# padding
if data.shape[0]%TIME_STEP!=0.0:
  padd_data = np.zeros((TIME_STEP*(data.shape[0]//TIME_STEP+1)+1,data.shape[1]))
  print('added {zeros} zeros'.format(zeros=padd_data.shape[0]-data.shape[0]))
  padd_data[padd_data.shape[0]-data.shape[0]:,:] = data
else:
  padd_data = np.zeros((data.shape[0]+1,data.shape[1]))
  print('added {zeros} zero'.format(zeros=1))
  padd_data[1:,:] = data
data = padd_data.copy()
del padd_data
data.shape

added 1 zero


(453633, 10)

In [ ]:
453633-453376

257

In [ ]:
max_val = data.shape[0]- 1 - 255
indices = np.arange(max_val)
max(indices)

453376

In [ ]:
453376+256

453632

In [ ]:
sub_data = data[5:5+256,:]
mx1 = np.max(sub_data, axis= 0)
sub_data = data[9:9+256,:]
mx2 = np.max(sub_data, axis= 0)

In [ ]:
mx1

array([4.48539000e+03, 4.48539000e+03, 4.48539000e+03, 4.48539000e+03,
       1.98580650e+01, 8.47179981e+04, 7.70000000e+01, 1.31385010e+01,
       5.66677835e+04, 8.67317144e+03])

In [ ]:
mx2

array([4.48539000e+03, 4.48539000e+03, 4.48539000e+03, 4.48539000e+03,
       1.98580650e+01, 8.47179981e+04, 7.70000000e+01, 1.31385010e+01,
       5.66677835e+04, 8.71576180e+03])

In [ ]:
a = mx1.copy()
a[np.where(mx2>mx1)] = mx2[np.where(mx2>mx1)]
a

array([4.48539000e+03, 4.48539000e+03, 4.48539000e+03, 4.48539000e+03,
       1.98580650e+01, 8.47179981e+04, 7.70000000e+01, 1.31385010e+01,
       5.66677835e+04, 8.71576180e+03])

In [ ]:
def create_tfrecs(path_to_csv ,save_path = '/content/data'):
  name = path_to_csv.split('/')[-1]
  name = name.split('.csv')[0]

  data = pd.read_csv(path_to_csv)
  data = data.drop(columns=['Close-time', 'Open-time'])
  data = np.array(data)
  # padding
  if data.shape[0]%TIME_STEP!=0.0:
    padd_data = np.zeros((TIME_STEP*(data.shape[0]//TIME_STEP+1)+1,data.shape[1]))
    padd_data[padd_data.shape[0]-data.shape[0]:,:] = data
  else:
    padd_data = np.zeros((data.shape[0]+1,data.shape[1]))
    padd_data[1:,:] = data
  data = padd_data.copy()
  del padd_data

  # train val indices
  max_idx = data.shape[0]- 1 - 255
  indices = np.arange(max_idx)
  train_idx, val_idx = train_test_split(indices, test_size=VAL_SIZE, random_state=40)
  train_idx = sorted(train_idx.tolist())
  val_idx = sorted(val_idx.tolist())

  max_values = np.zeros((NUM_FEATURES,))
  num_positives= 0
  with tf.io.TFRecordWriter(os.path.join(save_path, name + '_train.tfrec')) as writer:
    for i in train_idx:
      sub_data = data[i:i+256,:]

      mx1 = np.max(sub_data, axis= 0)
      max_values[np.where(mx1>max_values)] = mx1[np.where(mx1>max_values)]

      label = 0
      if data[i+256,3]> data[i+256-1,3] :
        label = 1
        num_positives += 1

      example = serialize_example(sub_data.reshape(-1,1), label)
      writer.write(example)
  print('Class 1:',num_positives )
  num_positives = 0
  with tf.io.TFRecordWriter(os.path.join(save_path, name + '_val.tfrec')) as writer:
    for i in val_idx:
      sub_data = data[i:i+256,:]
      label = 0
      if data[i+256,3]> data[i+256-1,3] :
        label = 1
        num_positives += 1

      example = serialize_example(sub_data.reshape(-1,1), label)
      writer.write(example)
  print('Class 1:VAL:',num_positives )
  return len(train_idx), len(val_idx), max_values

In [ ]:
int(2.54)

2

In [ ]:
def create_tfrecs(path_to_csv ,save_path = '/content/data'):
  name = path_to_csv.split('/')[-1]
  name = name.split('.csv')[0]

  data = pd.read_csv(path_to_csv)
  data = data.drop(columns=['Close-time', 'Open-time'])
  data = np.array(data)
  # padding
  if data.shape[0]%TIME_STEP!=0.0:
    padd_data = np.zeros((TIME_STEP*(data.shape[0]//TIME_STEP+1)+1,data.shape[1]))
    padd_data[padd_data.shape[0]-data.shape[0]:,:] = data
  else:
    padd_data = np.zeros((data.shape[0]+1,data.shape[1]))
    padd_data[1:,:] = data
  data = padd_data.copy()
  del padd_data

  # train val indices
  max_idx = data.shape[0]- 1 - 255
  train_idx = np.arange(start=0, stop=int(0.7*max_idx))
  train_idx = train_idx.tolist()
  val_idx = np.arange(start=int(0.7*max_idx), stop=max_idx)
  val_idx = val_idx.tolist()

  max_values = np.zeros((NUM_FEATURES,))
  max_labels = 0.0
  num_positives= 0
  with tf.io.TFRecordWriter(os.path.join(save_path, name + '_train.tfrec')) as writer:
    for i in train_idx:
      sub_data = data[i:i+256,:]

      mx1 = np.max(sub_data, axis= 0)
      max_values[np.where(mx1>max_values)] = mx1[np.where(mx1>max_values)]

      label = data[i+256,3]
      if label>max_labels:
        max_labels = label
      #if data[i+256,3]> data[i+256-1,3] :
      #  label = 1
      #  num_positives += 1

      example = serialize_example(sub_data.reshape(-1,1), label)
      writer.write(example)
  print('Class 1:',num_positives )
  num_positives = 0
  with tf.io.TFRecordWriter(os.path.join(save_path, name + '_val.tfrec')) as writer:
    for i in val_idx:
      sub_data = data[i:i+256,:]
      #label = 0
      #if data[i+256,3]> data[i+256-1,3] :
      #  label = 1
      #  num_positives += 1
      label = data[i+256,3]
      example = serialize_example(sub_data.reshape(-1,1), label)
      writer.write(example)
  print('Class 1:VAL:',num_positives )
  return len(train_idx), len(val_idx), max_values,max_labels

In [ ]:
path_to_csv=  '/content/drive/MyDrive/5min/BTCUSDT.csv'
num_train_samples, num_val_samples, max_values,max_labels = create_tfrecs(path_to_csv ,save_path = '/content')

Class 1: 0
Class 1:VAL: 0


In [ ]:
max_values

array([1.97620000e+04, 1.97986800e+04, 1.97290300e+04, 1.97620000e+04,
       8.27717230e+03, 7.78004571e+07, 5.83840000e+04, 4.95984888e+03,
       3.96974091e+07, 7.99473710e+04])

In [ ]:
num_train_samples

317363

In [ ]:
max_labels

19762.0

In [ ]:
num_val_samples

136014

In [ ]:
max_labels

19762.0

In [ ]:
num_train_samples = 353634

In [ ]:
176605/num_train_samples

0.4994005101319443

In [ ]:
num_val_samples = 99743

In [ ]:
49710/num_val_samples

0.4983808387556019

In [ ]:
# Create a dictionary describing the features.
feature_description = {
    'data': tf.io.FixedLenFeature([TIME_STEP*NUM_FEATURES], tf.float32, default_value=[0.0]*256*NUM_FEATURES),
    'target': tf.io.FixedLenFeature([1], tf.float32, default_value=[0.0]),
  }

def _parse_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def preprocess(features,effesionet=True):
  data = features['data']
  #data = tf.io.decode_raw(data, tf.int16)
  label = features['target']
  #label = tf.io.decode_raw(label, tf.int16)
  return data, label


def normalize(data ,label):
  #standard
  data = tf.cast(data, tf.float32)
  scale = tf.constant(max_values.astype(np.float32))

  label = tf.cast(label, tf.float32)
  data /= scale
  label /= tf.constant(np.float32(max_labels))
  return data , label

@tf.function
def augment(data,label):

  return data , label

def Resize(data,label):
  data = tf.reshape(data, [TIME_STEP,NUM_FEATURES])
  #label = tf.reshape(label, [256,1])
  return data , label


In [ ]:
ds_train = tf.data.TFRecordDataset(['/content/BTCUSDT_train.tfrec'])
ds_train = ds_train.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(100)
ds_train  =  ds_train.map(preprocess)#.map(agment)
ds_train  = ds_train.map(Resize).map(normalize).batch(BATCH_SIZE,drop_remainder=True).repeat()

In [ ]:
ds_val = tf.data.TFRecordDataset(['/content/BTCUSDT_val.tfrec'])
ds_val = ds_val.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val  =  ds_val.map(preprocess)#.map(agment)
ds_val  = ds_val.map(Resize).map(normalize).batch(BATCH_SIZE,drop_remainder=True).repeat()

In [ ]:
data1, label = next(iter(ds_val))
data1.shape

TensorShape([64, 256, 10])

In [ ]:
label

<tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[0.581301  ],
       [0.58146745],
       [0.58164406],
       [0.5825782 ],
       [0.5829511 ],
       [0.5824173 ],
       [0.58233273],
       [0.58183026],
       [0.5817726 ],
       [0.5817792 ],
       [0.5817589 ],
       [0.58195376],
       [0.58178324],
       [0.58141434],
       [0.58118916],
       [0.5810803 ],
       [0.581848  ],
       [0.58136624],
       [0.5810176 ],
       [0.5816638 ],
       [0.58089566],
       [0.5811264 ],
       [0.58107275],
       [0.580424  ],
       [0.5809316 ],
       [0.5813875 ],
       [0.5809776 ],
       [0.5802368 ],
       [0.58124787],
       [0.5805663 ],
       [0.5820205 ],
       [0.58135813],
       [0.5806988 ],
       [0.5809776 ],
       [0.5803881 ],
       [0.58010167],
       [0.57995397],
       [0.58043164],
       [0.5801913 ],
       [0.5799641 ],
       [0.5802419 ],
       [0.58072263],
       [0.5804073 ],
       [0.58055663],
       [0.58101004],
      

In [ ]:
l

In [ ]:
a = tf.data.TFRecordDataset(['/content/BTCUSDT_val.tfrec'])
a = a.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
a  =  a.map(preprocess)#.map(agment)
a  = a.map(Resize).map(normalize).batch(BATCH_SIZE,drop_remainder=True)
mx = 0.0
for b in a:
  d, l = b
  l = l.numpy()
  m = np.max(l)
  if m>mx:
    mx=m
print(mx)

3.4781022


In [ ]:
from tensorflow.keras.activations import selu, relu

class BB (tf.keras.layers.Layer):
  def __init__(self,
               filters = 64,
               kernel_size = 9):
    super(BB, self).__init__()

    self.filters = filters
    self.kernel_size = kernel_size

    self.conv1 = tf.keras.layers.Conv1D(filters, kernel_size, activation=None, padding='same')

    self.bn1 = tf.keras.layers.BatchNormalization()
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.bn3 = tf.keras.layers.BatchNormalization()
    self.conv2 = tf.keras.layers.Conv1D(filters, kernel_size, activation=None, padding='same')


  def call(self, z, training = True):
    x = self.conv1(z)
    x = self.bn1(x, training = training)
    x = selu(x)

    x = self.conv2(x)
    x = self.bn2(x, training = training)
    x = selu(x)
    x = tf.keras.layers.Add()([x, z])

    x = self.bn3(x, training = training)
    x = selu(x)
    return x

In [ ]:
def tf_max_pool_with_masks(x,
                           ksize = [1,1,2,1],
                           strides = [1,1,2,1],
                           padding = "VALID",
                           data_format='NHWC',
                           output_dtype=tf.dtypes.int64,
                           include_batch_in_index=True):
  """
    x = input tensor of size BTC, B=Batch, T:Time step, C:Channel
    ksize: kernel size for each dim
    strides: strides of each dim
    data_format:channel last or not
    output_dtype: int64 or int32
    include_batch_in_index:An optional boolean. Defaults to True. Whether to include batch dimension in flattened index of argmax.
  """
  input = tf.reshape(x, shape=(x.shape[0], 1, x.shape[1],x.shape[2]))
  output, argmax = tf.nn.max_pool_with_argmax(input,
                                              ksize = [1,1,2,1],
                                              strides = [1,1,2,1],
                                              padding = "VALID",
                                              data_format='NHWC',
                                              output_dtype=tf.dtypes.int64,
                                              include_batch_in_index=True
                                              )

  #a = tf.reshape(output, shape=(-1,1))
  indices = tf.reshape(argmax, shape=(-1,1))
  input_flatten = tf.reshape(input, shape=(-1,1))
  tensor = tf.zeros_like(input_flatten, dtype=tf.float32)
  updates = tf.ones_like(indices, dtype=tf.float32)
  mask = tf.tensor_scatter_nd_update(tensor, indices, updates)
  mask = tf.reshape(mask, shape=x.get_shape().as_list())
  output = tf.reshape(output, shape=(output.shape[0],output.shape[2],output.shape[3]))
  return output, mask

In [ ]:
from tensorflow import keras

class eUNet1D (keras.Model):
  def __init__(self,
               kernel_size = 9,
               strides = 2):
    super(eUNet1D, self).__init__()
    self.kernel_size        = kernel_size
    self.strides            = strides
    self.conv1_0 = tf.keras.layers.Conv1D(64, kernel_size, activation='selu', padding='same')
    self.bb1_1 = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2 = BB(filters = 64, kernel_size = self.kernel_size)

    #self.down2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv2_0 = tf.keras.layers.Conv1D(128, kernel_size, activation='selu', padding='same')
    self.bb2_1 = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb2_2 = BB(filters = 128, kernel_size = self.kernel_size)

    #self.down3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv3_0 = tf.keras.layers.Conv1D(256, kernel_size, activation='selu', padding='same')
    self.bb3_1 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_2 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_3 = BB(filters = 256, kernel_size = self.kernel_size)

    #self.down4 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv4_0 = tf.keras.layers.Conv1D(512, kernel_size, activation='selu', padding='same')
    self.bb4_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_3 = BB(filters = 512, kernel_size = self.kernel_size)

    #self.down5 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.bb5_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3 = BB(filters = 512, kernel_size = self.kernel_size)

    self.bb5_1_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.up5 =  tf.keras.layers.Conv1DTranspose(filters = 512, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv4_0_ = tf.keras.layers.Conv1D(256, kernel_size, activation='selu', padding='same')
    self.bb4_1_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_2_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_3_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.up4 =  tf.keras.layers.Conv1DTranspose(filters = 256, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv3_0_ = tf.keras.layers.Conv1D(128, kernel_size, activation='selu', padding='same')
    self.bb3_1_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_2_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_3_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.up3 =  tf.keras.layers.Conv1DTranspose(filters = 128, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv2_0_ = tf.keras.layers.Conv1D(64, kernel_size, activation='selu', padding='same')
    self.bb2_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb2_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.up2 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='valid')

    self.bb1_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    #self.up1 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='same')
    self.conv_out = tf.keras.layers.Conv1D(16, 1, activation='selu')#256*16

    #self.conv_out1 = tf.keras.layers.Conv1D(8, self.strides, activation='relu', padding='same')
    #self.f = tf.keras.layers.Flatten()
    '''
    self.bn = tf.keras.layers.BatchNormalization(axis=-1)
    self.gap = tf.keras.layers.GlobalAveragePooling1D()
    self.do = tf.keras.layers.Dropout(0.2)
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*4
    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    self.conv_out2_2 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=16, padding='valid')#1*4

    self.conv_out3 = tf.keras.layers.Conv1D(1, self.strides, activation='relu', padding='same')#1*1
    self.rs = tf.keras.layers.Reshape((BATCH_SIZE, 1))



  def call(self, z, training = True):
    """
      z: (btach, time steps, channels)--> input
    """
    #######################################
    ############ Row 1 ####################
    #######################################
    x = self.conv1_0(z)#(None, 256, 64)
    x = self.bb1_1(x) #(None, 256, 64)
    x1 = self.bb1_2(x) #(None, 256, 64)

    #######################################
    ############ Row 2 ####################
    #######################################
    x, mask2 = tf_max_pool_with_masks(x1) #(None, 128, 64)
    x = self.conv2_0(x)
    x = self.bb2_1(x) #(None, 128, 128)
    x2 = self.bb2_2(x) #(None, 128, 128)

    #######################################
    ############ Row 3 ####################
    #######################################
    x, mask3 = tf_max_pool_with_masks(x2) #(None, 64, 128)
    x = self.conv3_0(x)
    x = self.bb3_1(x) #(None, 64, 256)
    x = self.bb3_2(x) #(None, 64, 256)
    x3 = self.bb3_3(x) #(None, 64, 256)

    #######################################
    ############ Row 4 ####################
    #######################################
    x, mask4 = tf_max_pool_with_masks(x3) #(None, 32, 256)
    x = self.conv4_0(x)
    x = self.bb4_1(x) #(None, 32, 512)
    x = self.bb4_2(x) #(None, 32, 512)
    x4 = self.bb4_3(x) #(None, 32, 512)

    #######################################
    ############ Row 5 ####################
    #######################################
    x, mask5 = tf_max_pool_with_masks(x4) #(None, 16, 512)
    x = self.bb5_1(x) #(None, 16, 512)
    x = self.bb5_2(x) #(None, 16, 512)
    x = self.bb5_3(x) #(None, 16, 512)

    #######################################
    ############ Row 5_1 ##################
    #######################################
    x = self.bb5_1_(x) #(None, 16, 512)
    x = self.bb5_2_(x) #(None, 16, 512)
    x = self.bb5_3_(x) #(None, 16, 512)
    x = self.up5(x) #(None, 32, 512)
    x = tf.math.multiply(x, mask5) #(None, 32, 512)

    #######################################
    ############ Row 4_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x4]) #(None, 32, 512)
    x = self.conv4_0_(x)
    x = self.bb4_1_(x) #(None, 32, 256)
    x = self.bb4_2_(x) #(None, 32, 256)
    x = self.bb4_3_(x) #(None, 32, 256)
    x = self.up4(x) #(None, 64, 256)
    x = tf.math.multiply(x, mask4) #(None, 64, 256)

    #######################################
    ############ Row 3_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x3]) #(None, 64, 256)
    x = self.conv3_0_(x)
    x = self.bb3_1_(x) #(None, 64, 128)
    x = self.bb3_2_(x) #(None, 64, 128)
    x = self.bb3_3_(x) #(None, 64, 128)
    x = self.up3(x) #(None, 128, 128)
    x = tf.math.multiply(x, mask3) #(None, 128, 128)

    #######################################
    ############ Row 2_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x2]) #(None, 128, 128)
    x = self.conv2_0_(x)
    x = self.bb2_1_(x) #(None, 128, 64)
    x = self.bb2_2_(x) #(None, 128, 64)
    x = self.up2(x) #(None, 256, 64)
    x = tf.math.multiply(x, mask2) #(None, 256, 64)

    #######################################
    ############ Row 1_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x1]) #(None, 256, 64)
    x = self.bb1_1_(x) #(None, 256, 64)
    x = self.bb1_2_(x) #(None, 256, 64)

    x = self.conv_out(x)#(None, 256, 16)
    '''
    x = self.bn(x)
    x = self.gap(x)
    x = self.do(x)
    x = self.fc(x)

    '''
    x = self.conv_out2_1(x)
    x = self.mp2(x)#(None, 16, 4)
    x = self.f(x)
    x = self.fc(x)
    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*4
    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')


    x = self.conv_out2_2(x)
    x = self.mp3(x)#(None, 1, 4)

    x = self.conv_out3(x)
    x = self.mp3(x)#(None, 1, 1)
    x = tf.reshape(x, (x.shape[0],1))
    '''

    return x

In [ ]:
from tensorflow import keras

class eUNet1D (keras.Model):
  def __init__(self,
               kernel_size = 9,
               strides = 2):
    super(eUNet1D, self).__init__()
    self.kernel_size        = kernel_size
    self.strides            = strides
    self.conv1_0 = tf.keras.layers.Conv1D(64, kernel_size, activation='selu', padding='same')
    self.bb1_1 = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2 = BB(filters = 64, kernel_size = self.kernel_size)

    #self.down2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv2_0 = tf.keras.layers.Conv1D(128, kernel_size, activation='selu', padding='same')
    self.bb2_1 = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb2_2 = BB(filters = 128, kernel_size = self.kernel_size)

    #self.down3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv3_0 = tf.keras.layers.Conv1D(256, kernel_size, activation='selu', padding='same')
    self.bb3_1 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_2 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_3 = BB(filters = 256, kernel_size = self.kernel_size)

    #self.down4 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv4_0 = tf.keras.layers.Conv1D(512, kernel_size, activation='selu', padding='same')
    self.bb4_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_3 = BB(filters = 512, kernel_size = self.kernel_size)

    #self.down5 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.bb5_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3 = BB(filters = 512, kernel_size = self.kernel_size)

    self.bb5_1_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.up5 =  tf.keras.layers.Conv1DTranspose(filters = 512, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv4_0_ = tf.keras.layers.Conv1D(256, kernel_size, activation='selu', padding='same')
    self.bb4_1_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_2_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_3_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.up4 =  tf.keras.layers.Conv1DTranspose(filters = 256, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv3_0_ = tf.keras.layers.Conv1D(128, kernel_size, activation='selu', padding='same')
    self.bb3_1_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_2_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_3_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.up3 =  tf.keras.layers.Conv1DTranspose(filters = 128, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv2_0_ = tf.keras.layers.Conv1D(64, kernel_size, activation='selu', padding='same')
    self.bb2_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb2_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.up2 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='valid')

    self.bb1_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    #self.up1 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='same')
    self.conv_out = tf.keras.layers.Conv1D(64, 1, activation='selu')#256*16

    #self.conv_out1 = tf.keras.layers.Conv1D(8, self.strides, activation='relu', padding='same')
    #self.f = tf.keras.layers.Flatten()
    '''
    self.bn = tf.keras.layers.BatchNormalization(axis=-1)
    self.gap = tf.keras.layers.GlobalAveragePooling1D()
    self.do = tf.keras.layers.Dropout(0.2)
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(32, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*32

    self.conv_out3_1 = tf.keras.layers.Conv1D(32, self.strides, activation='relu', padding='same')
    self.mp3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#4*32

    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')
    '''
    self.conv_out2_2 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=16, padding='valid')#1*4

    self.conv_out3 = tf.keras.layers.Conv1D(1, self.strides, activation='relu', padding='same')#1*1
    self.rs = tf.keras.layers.Reshape((BATCH_SIZE, 1))
    '''


  def call(self, z, training = True):
    """
      z: (btach, time steps, channels)--> input
    """
    #######################################
    ############ Row 1 ####################
    #######################################
    x = self.conv1_0(z)#(None, 256, 64)
    x = self.bb1_1(x) #(None, 256, 64)
    x1 = self.bb1_2(x) #(None, 256, 64)

    #######################################
    ############ Row 2 ####################
    #######################################
    x, mask2 = tf_max_pool_with_masks(x1) #(None, 128, 64)
    x = self.conv2_0(x)
    x = self.bb2_1(x) #(None, 128, 128)
    x2 = self.bb2_2(x) #(None, 128, 128)

    #######################################
    ############ Row 3 ####################
    #######################################
    x, mask3 = tf_max_pool_with_masks(x2) #(None, 64, 128)
    x = self.conv3_0(x)
    x = self.bb3_1(x) #(None, 64, 256)
    x = self.bb3_2(x) #(None, 64, 256)
    x3 = self.bb3_3(x) #(None, 64, 256)

    #######################################
    ############ Row 4 ####################
    #######################################
    x, mask4 = tf_max_pool_with_masks(x3) #(None, 32, 256)
    x = self.conv4_0(x)
    x = self.bb4_1(x) #(None, 32, 512)
    x = self.bb4_2(x) #(None, 32, 512)
    x4 = self.bb4_3(x) #(None, 32, 512)

    #######################################
    ############ Row 5 ####################
    #######################################
    x, mask5 = tf_max_pool_with_masks(x4) #(None, 16, 512)
    x = self.bb5_1(x) #(None, 16, 512)
    x = self.bb5_2(x) #(None, 16, 512)
    x = self.bb5_3(x) #(None, 16, 512)

    #######################################
    ############ Row 5_1 ##################
    #######################################
    x = self.bb5_1_(x) #(None, 16, 512)
    x = self.bb5_2_(x) #(None, 16, 512)
    x = self.bb5_3_(x) #(None, 16, 512)
    x = self.up5(x) #(None, 32, 512)
    x = tf.math.multiply(x, mask5) #(None, 32, 512)

    #######################################
    ############ Row 4_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x4]) #(None, 32, 512)
    x = self.conv4_0_(x)
    x = self.bb4_1_(x) #(None, 32, 256)
    x = self.bb4_2_(x) #(None, 32, 256)
    x = self.bb4_3_(x) #(None, 32, 256)
    x = self.up4(x) #(None, 64, 256)
    x = tf.math.multiply(x, mask4) #(None, 64, 256)

    #######################################
    ############ Row 3_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x3]) #(None, 64, 256)
    x = self.conv3_0_(x)
    x = self.bb3_1_(x) #(None, 64, 128)
    x = self.bb3_2_(x) #(None, 64, 128)
    x = self.bb3_3_(x) #(None, 64, 128)
    x = self.up3(x) #(None, 128, 128)
    x = tf.math.multiply(x, mask3) #(None, 128, 128)

    #######################################
    ############ Row 2_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x2]) #(None, 128, 128)
    x = self.conv2_0_(x)
    x = self.bb2_1_(x) #(None, 128, 64)
    x = self.bb2_2_(x) #(None, 128, 64)
    x = self.up2(x) #(None, 256, 64)
    x = tf.math.multiply(x, mask2) #(None, 256, 64)

    #######################################
    ############ Row 1_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x1]) #(None, 256, 64)
    x = self.bb1_1_(x) #(None, 256, 64)
    x = self.bb1_2_(x) #(None, 256, 64)

    x = self.conv_out(x)#(None, 256, 16)
    '''
    x = self.bn(x)
    x = self.gap(x)
    x = self.do(x)
    x = self.fc(x)

    '''
    x = self.conv_out2_1(x)
    x = self.mp2(x)#(None, 16, 4)
    x = self.f(x)
    x = self.fc(x)
    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*4
    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')


    x = self.conv_out2_2(x)
    x = self.mp3(x)#(None, 1, 4)

    x = self.conv_out3(x)
    x = self.mp3(x)#(None, 1, 1)
    x = tf.reshape(x, (x.shape[0],1))
    '''

    return x

In [ ]:
from tensorflow import keras

class eUNet1D (keras.Model):
  def __init__(self,
               kernel_size = 9,
               strides = 2):
    super(eUNet1D, self).__init__()
    self.kernel_size        = kernel_size
    self.strides            = strides
    self.conv1_0 = tf.keras.layers.Conv1D(64, kernel_size, activation='relu', padding='same')
    self.bb1_1 = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2 = BB(filters = 64, kernel_size = self.kernel_size)

    #self.down2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv2_0 = tf.keras.layers.Conv1D(128, kernel_size, activation='relu', padding='same')
    self.bb2_1 = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb2_2 = BB(filters = 128, kernel_size = self.kernel_size)

    #self.down3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv3_0 = tf.keras.layers.Conv1D(256, kernel_size, activation='relu', padding='same')
    self.bb3_1 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_2 = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb3_3 = BB(filters = 256, kernel_size = self.kernel_size)

    #self.down4 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.conv4_0 = tf.keras.layers.Conv1D(512, kernel_size, activation='relu', padding='same')
    self.bb4_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb4_3 = BB(filters = 512, kernel_size = self.kernel_size)

    #self.down5 = tf.keras.layers.MaxPool1D(pool_size=2, strides=self.strides, padding='valid')
    self.bb5_1 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2 = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3 = BB(filters = 512, kernel_size = self.kernel_size)

    self.bb5_1_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_2_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.bb5_3_ = BB(filters = 512, kernel_size = self.kernel_size)
    self.up5 =  tf.keras.layers.Conv1DTranspose(filters = 512, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv4_0_ = tf.keras.layers.Conv1D(256, kernel_size, activation='relu', padding='same')
    self.bb4_1_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_2_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.bb4_3_ = BB(filters = 256, kernel_size = self.kernel_size)
    self.up4 =  tf.keras.layers.Conv1DTranspose(filters = 256, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv3_0_ = tf.keras.layers.Conv1D(128, kernel_size, activation='relu', padding='same')
    self.bb3_1_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_2_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.bb3_3_ = BB(filters = 128, kernel_size = self.kernel_size)
    self.up3 =  tf.keras.layers.Conv1DTranspose(filters = 128, kernel_size = 2, strides=self.strides, padding='valid')

    self.conv2_0_ = tf.keras.layers.Conv1D(64, kernel_size, activation='relu', padding='same')
    self.bb2_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb2_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.up2 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='valid')

    self.bb1_1_ = BB(filters = 64, kernel_size = self.kernel_size)
    self.bb1_2_ = BB(filters = 64, kernel_size = self.kernel_size)
    #self.up1 =  tf.keras.layers.Conv1DTranspose(filters = 64, kernel_size = 2, strides=self.strides, padding='same')
    self.conv_out = tf.keras.layers.Conv1D(16, 1, activation='relu')#256*16

    #self.conv_out1 = tf.keras.layers.Conv1D(8, self.strides, activation='relu', padding='same')
    #self.f = tf.keras.layers.Flatten()
    '''
    self.bn = tf.keras.layers.BatchNormalization(axis=-1)
    self.gap = tf.keras.layers.GlobalAveragePooling1D()
    self.do = tf.keras.layers.Dropout(0.2)
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    '''
    self.lstm1 = tf.keras.layers.LSTM(units = 60, activation = 'relu', return_sequences = True)
    self.lstm2 = tf.keras.layers.LSTM(units = 80, activation = 'relu', return_sequences = True)
    self.lstm3 = tf.keras.layers.LSTM(units = 120, activation = 'relu')

    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='relu')
    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(32, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*32

    self.conv_out3_1 = tf.keras.layers.Conv1D(32, self.strides, activation='relu', padding='same')
    self.mp3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#4*32

    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')
    '''
    '''
    self.conv_out2_2 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=16, padding='valid')#1*4

    self.conv_out3 = tf.keras.layers.Conv1D(1, self.strides, activation='relu', padding='same')#1*1
    self.rs = tf.keras.layers.Reshape((BATCH_SIZE, 1))
    '''


  def call(self, z, training = True):
    """
      z: (btach, time steps, channels)--> input
    """
    #######################################
    ############ Row 1 ####################
    #######################################
    inputs = z
    x = self.conv1_0(z)#(None, 256, 64)
    x = self.bb1_1(x) #(None, 256, 64)
    x1 = self.bb1_2(x) #(None, 256, 64)

    #######################################
    ############ Row 2 ####################
    #######################################
    x, mask2 = tf_max_pool_with_masks(x1) #(None, 128, 64)
    x = self.conv2_0(x)
    x = self.bb2_1(x) #(None, 128, 128)
    x2 = self.bb2_2(x) #(None, 128, 128)

    #######################################
    ############ Row 3 ####################
    #######################################
    x, mask3 = tf_max_pool_with_masks(x2) #(None, 64, 128)
    x = self.conv3_0(x)
    x = self.bb3_1(x) #(None, 64, 256)
    x = self.bb3_2(x) #(None, 64, 256)
    x3 = self.bb3_3(x) #(None, 64, 256)

    #######################################
    ############ Row 4 ####################
    #######################################
    x, mask4 = tf_max_pool_with_masks(x3) #(None, 32, 256)
    x = self.conv4_0(x)
    x = self.bb4_1(x) #(None, 32, 512)
    x = self.bb4_2(x) #(None, 32, 512)
    x4 = self.bb4_3(x) #(None, 32, 512)

    #######################################
    ############ Row 5 ####################
    #######################################
    x, mask5 = tf_max_pool_with_masks(x4) #(None, 16, 512)
    x = self.bb5_1(x) #(None, 16, 512)
    x = self.bb5_2(x) #(None, 16, 512)
    x = self.bb5_3(x) #(None, 16, 512)

    #######################################
    ############ Row 5_1 ##################
    #######################################
    x = self.bb5_1_(x) #(None, 16, 512)
    x = self.bb5_2_(x) #(None, 16, 512)
    x = self.bb5_3_(x) #(None, 16, 512)
    x = self.up5(x) #(None, 32, 512)
    x = tf.math.multiply(x, mask5) #(None, 32, 512)

    #######################################
    ############ Row 4_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x4]) #(None, 32, 512)
    x = self.conv4_0_(x)
    x = self.bb4_1_(x) #(None, 32, 256)
    x = self.bb4_2_(x) #(None, 32, 256)
    x = self.bb4_3_(x) #(None, 32, 256)
    x = self.up4(x) #(None, 64, 256)
    x = tf.math.multiply(x, mask4) #(None, 64, 256)

    #######################################
    ############ Row 3_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x3]) #(None, 64, 256)
    x = self.conv3_0_(x)
    x = self.bb3_1_(x) #(None, 64, 128)
    x = self.bb3_2_(x) #(None, 64, 128)
    x = self.bb3_3_(x) #(None, 64, 128)
    x = self.up3(x) #(None, 128, 128)
    x = tf.math.multiply(x, mask3) #(None, 128, 128)

    #######################################
    ############ Row 2_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x2]) #(None, 128, 128)
    x = self.conv2_0_(x)
    x = self.bb2_1_(x) #(None, 128, 64)
    x = self.bb2_2_(x) #(None, 128, 64)
    x = self.up2(x) #(None, 256, 64)
    x = tf.math.multiply(x, mask2) #(None, 256, 64)

    #######################################
    ############ Row 1_1 ##################
    #######################################
    x = tf.keras.layers.Add()([x, x1]) #(None, 256, 64)
    x = self.bb1_1_(x) #(None, 256, 64)
    x = self.bb1_2_(x) #(None, 256, 64)

    x = self.conv_out(x)#(None, 256, 16)
    '''
    x = self.bn(x)
    x = self.gap(x)
    x = self.do(x)
    x = self.fc(x)

    '''
    #print(x.shape)

    x = tf.concat([x, inputs], -1)
    x = self.lstm1(x)
    x = self.lstm2(x)
    x = self.lstm3(x)
    #x = self.f(x)
    x = self.fc(x)
    '''
    self.conv_out2_1 = tf.keras.layers.Conv1D(4, self.strides, activation='relu', padding='same')
    self.mp2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=8, padding='valid')#32*4
    self.f = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(1, activation='sigmoid')


    x = self.conv_out2_2(x)
    x = self.mp3(x)#(None, 1, 4)

    x = self.conv_out3(x)
    x = self.mp3(x)#(None, 1, 1)
    x = tf.reshape(x, (x.shape[0],1))
    '''

    return x

In [ ]:
x = tf.constant(1.0, dtype=tf.float32, shape=(7,256,10))*np.random.rand()
model = eUNet1D(kernel_size = 9)
temp_out = model(x, training = False)
temp_out.shape

TensorShape([7, 1])

In [ ]:
temp_out

<tf.Tensor: shape=(7, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [ ]:
TRAIN_STEPS = num_train_samples//BATCH_SIZE+1
VAL_STEPS = num_val_samples//BATCH_SIZE

In [ ]:
def cce_with_label_smoothing(y_true,y_pred):
  return tf.keras.losses.binary_crossentropy(y_true,y_pred,label_smoothing=0.2)

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
lr_sc = tf.keras.optimizers.schedules.ExponentialDecay(1e-4, TRAIN_STEPS ,decay_rate=0.96)
optimizer = tf.keras.optimizers.Adam(1e-3,beta_1=0.9,beta_2=0.999)
model.compile(loss= 'mse',
              optimizer = optimizer,
              metrics=['mae'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/drive/MyDrive/models/saved_model-lr1e-3-3lstm-{epoch:02d}-{val_loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',
                             save_weights_only=True,
                             verbose=1, save_best_only=True,
                             mode='min')
#callbacks_list = [ checkpoint,tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,) ]
callbacks_list = [ checkpoint]

history = model.fit(ds_train,
                    epochs= EPOCHS,
                    validation_data = ds_val,
                    steps_per_epoch= TRAIN_STEPS,
                    validation_steps = VAL_STEPS,
                    callbacks = callbacks_list)